# DataLab 2: Voetbal & Data Science <a name='begin'></a>

<br>Groepsnummer: 3B
<br>Sprintnummer: 3 <br><br> Leden:
<br>    **Busse Heemskerk** 22122877 
<br>    **Isa Dijkstra**                             22119485
<br> **Jacco van den Dobbelsteen**  17100976
<br>    **Johan Heskes**                        22132376
<br>    **Tim Oosterling**                        22078800

## Table of Contents

1. [Importeren van de dataset in Python](#dataset)
2. [Sprint 2](#spr2)
    1. [Beschrijven van de tabellen](#tabellen)
    2. [Bekijken van de data van Heracles Almelo](#hadata)
    3. [Eigen functies; Statistieken per seizoen bepalen](#define1)
        1. [Het runnen van de functies](#run1)
3. [Sprint 3](#spr3)
    1. [Eigen functies: Prestaties van Heracles Almelo](#prestatie)
        1. [Bekijken van de prestaties](#prebekijk)
    2. [Onderzoek teameigenschappen](#team)
        1. [kolom_seizoen](#kolmaken)
    3. [Visualisaties](#visu1)
        1. [Tonen van de grafieken](#run3)
    

## Importeren van de dataset in Python <a name='dataset'></a>

Voordat er gecodeerd wordt, worden de mogelijk nodige libraries voor de opdrachten geïmporteerd

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from functools import wraps

Eerst word er een connectie gemaakt met de SQL-database door middel van sqlite3, vervolgens worden de tabellen per stuk geïmporteerd door middel van Pandas.

In [ ]:
connect = sqlite3.connect('database.sqlite')

country = pd.read_sql_query("SELECT * FROM Country", connect)
league = pd.read_sql_query("SELECT * FROM League", connect)
match = pd.read_sql_query("SELECT * FROM Match", connect)
player = pd.read_sql_query("SELECT * FROM Player", connect)
player_attributes = pd.read_sql_query("SELECT * FROM Player_attributes", connect)
team = pd.read_sql_query("SELECT * FROM Team", connect)
team_attributes = pd.read_sql_query("SELECT * FROM Team_attributes", connect)

Vervolgens wordt de optie om de kolommen in te korten uitgezet, daarna worden alle kolommen getoont.

In [ ]:
pd.set_option("display.max_columns", None)
display(country, league, match, player, player_attributes, team, team_attributes)

[Terug naar het begin](#begin)

# Sprint 2 <a name='spr2' ></a>

## Beschrijven van de tabellen <a name='tabellen'></a>

Hier is de beschrijving van elke tabel gegeven, waarbij de naam en de beschrijving te zien zijn. De beschrijving bevat wat er in de tabel te vinden is.

| Tabel naam | Tabel beschrijving 
|:-------------------|:---------------
Country | In deze tabel staan de namen van de landen en de daarbij horende ID's
League | In deze tabel staan de namen van de competities per land samen met landelijke ID
Match | In deze tabel staat alle informatie die met wedstrijden te maken heeft en informatie over het wedden op de wedstrijden
Player | In deze tabel staat info over alle spelers, hun fifa ID, wanneer ze jarig zijn, hun lengte (in cm) en hun gewicht (in lbs) 
Player_attributes | In deze tabel staan de statistieken en rating van elke speler in de database 
Team | In deze tabel staat de id's van de club, de volledige club naam en de afkorting van de club
Team_attributes | In deze tabel zijn de statistieken van de teams in de database vermeld. zowel offensieve als defensieve statestieken zijn hierbij inbegrepen

[Terug naar het begin](#begin)

## Bekijken van de data van Heracles Almelo <a name='hadata'></a>

Eerst word er een cell gerunt om de team ID te vinden van onze club: Heracles Almelo

In [ ]:
team[team["team_long_name"] == "Heracles Almelo"]

De gegevens van het gekozen team staan getoond in de uitkomst van de onderstaande cell.

In [ ]:
team_id = team[team['team_long_name'] == 'Heracles Almelo'].iloc[0,1]
print('Team ID:', team_id)
print('Name:', team[team['team_long_name'] == 'Heracles Almelo'].iloc[0,3])
print('Short name:', team[team['team_long_name'] == 'Heracles Almelo'].iloc[0,4])
print('Country:', 'Netherlands')
print('League id:', country[country['name'] == 'Netherlands'].iloc[0,0])
print("League name:", league[league["id"] == 13274].iloc[0,2])

De gegevens van elke wedstrijd die het gekozen team heeft gespeeld per seizoen is hieronder getoond.

In [ ]:
display(match[(match['home_team_api_id'] == team_id) | (match['away_team_api_id'] == team_id)])

[Terug naar het begin](#begin)

## Eigen functies; Statistieken per seizoen bepalen <a name='define1'></a>

Hieronder zijn 4 functies gedefineerd:

    id_zoeker
    punten
    statistiek
    ranglijst
De uitleg van alle functies staat omschreven in de docstrings

### id_zoeker <a name='id_zoeker'></a>

In [ ]:
def id_zoeker(club):
    """
    Deze functie haalt het ID-nummer van de gegeven club op
    
    Parameters
    ----------
    club : str 
        De volledige naam van de club, waarvan je het ID wilt weten
    
    Returns
    ----------
    id : integer
        Het ID-nummer van de gekozen club
        
    """
    
    #Filteren
    id = team[team['team_long_name'] == club].iloc[0,1]
    return id

[Terug naar het begin](#begin)

### punten <a name='punten'></a>

In [ ]:
def punten(club, seizoen):
    """
    Deze functie maakt, op basis van een club en een seizoen, 2 kolommen aan. Eén over het aantal punten en één over 
    of de wedstrijd thuis of uit was voor het gekozen team.
    
    Parameters
    ----------
    club : str 
        De volledige naam van de club, waarvan je wilt weten hoeveel er gescoord is
    
    seizoen : str
        Het speel seizoen waar je de data van wilt weten
    
    Returns
    ----------
    wed : Pandas DataFrame
        Een dataframe gebaseerd op match, waarbij alleen de data van de aangegeven club en seizoen wordt getoond
    
    """
    
    #Aanmaken van een nieuwe gefilterde dataset en 2 lege lists
    team_id = id_zoeker(club)
    wed = match[((match['home_team_api_id'] == team_id) | (match['away_team_api_id'] == team_id)) & (match['season'] == seizoen)]
    punten_ = []
    tu = []
    
    #Beginnen van een list comprehension om de lijsten te vullen
    punten_ = [3 if row['home_team_api_id'] == team_id and row['home_team_goal'] > row['away_team_goal']
           else 1 if row['home_team_api_id'] == team_id and row['home_team_goal'] == row['away_team_goal']
           else 0 if row['home_team_api_id'] == team_id
           else 3 if row['away_team_goal'] > row['home_team_goal']
           else 1 if row['away_team_goal'] == row['home_team_goal']
           else 0 for _, row in wed.iterrows()]

    tu = ['thuis' if row['home_team_api_id'] == team_id else 'uit' for _, row in wed.iterrows()]
        
    #Aanmaken van copy en 2 nieuwe kolommen
    wed = wed.copy()
    wed.loc[:, 'punten'] = punten_
    wed.loc[:, 'thuis / uit'] = tu
    return wed

[Terug naar het begin](#begin)

### statistiek <a name='statistiek'></a>

In [ ]:
def statistiek(club, seizoen):
    """
    Deze functie laat de statistieken zien over de gegeven club in het gegeven seizoen. 
    
    Parameters
    ----------
    club : str 
        De volledige naam van de club, waarvan je de statistieken wilt weten
    
    seizoen : str
        Het speel seizoen waar je de data van wilt weten
    
    Returns
    ----------
    df : Pandas DataFrame
        Een dataframe waarin de naam, het aantal wedstrijden, het aantal punten, het gemiddelde aantal punten 
        per wedstrijd en de winst percentages staan
    
    """
    
    #Aanmaken van 2 aparte dataframes voor percentages
    t = punten(club, seizoen)[punten(club, seizoen)['thuis / uit'] == 'thuis']
    u = punten(club, seizoen)[punten(club, seizoen)['thuis / uit'] == 'uit']
    
    #Maken van een list met alle benodigde data voor het dataframe 
    stats = [[club,
            seizoen,
            len(punten(club, seizoen)),
            sum(punten(club, seizoen)['punten']),
            round(sum(punten(club, seizoen)['punten'] / len(punten(club, seizoen)['punten'])), 2),
            round(t[t['punten'] == 3]['punten'].count() / t['punten'].count() * 100, 2),
            round(t[t['punten'] == 1]['punten'].count() / t['punten'].count() * 100, 2),
            round(t[t['punten'] == 0]['punten'].count() / t['punten'].count() * 100, 2),
            round(u[u['punten'] == 3]['punten'].count() / u['punten'].count() * 100, 2),
            round(u[u['punten'] == 1]['punten'].count() / u['punten'].count() * 100, 2),
            round(u[u['punten'] == 0]['punten'].count() / u['punten'].count() * 100, 2)]]
    
    #Maken van het dataframe
    df = pd.DataFrame(stats, columns =['team',
                                       'seizoen',
                                       'wedstrijden', 
                                       'punten',
                                       'gemiddeld_punten',
                                       'thuis_win%',
                                       'thuis_gelijk%',
                                       'thuis_verlies%',
                                       'uit_win%',
                                       'uit_gelijk%',
                                       'uit_verlies%'])
       
    return df

[Terug naar het begin](#begin)

### ranglijst <a name='ranglijst'></a>

In [ ]:
load_screen = ["Welcome to this journey!..............................................",    
                       "Preparing to load dataset.............................................",    
                       "Checking for missing files...............................................",    
                       "Initializing data structures................................................",    
                       "Setting up parameters.....................................................",    
                       "Loading feature vectors................................................",    
                       "Applying pre-processing steps......................................",    
                       "Filtering out invalid samples.............................................",    
                       "Compiling statistics.......................................................",    
                       "Normalizing data...........................................................",    
                       "Building training and test sets..........................................",    
                       "Cross-validating models.................................................",   
                       "Fine-tuning hyperparameters..........................................",    
                       "Evaluating performance metrics....................................",    
                       "The dataset is almost loaded!........................................"]

def ranglijst(league_naam, seizoen):
    """
    Deze functie laat de ranglijst zien over de gegeven league in het gegeven seizoen. 
    
    Parameters
    ----------
    league_naam : str 
        De naam van de league waarvan je de ranglijst wilt bepalen
    
    seizoen : str
        Het speel seizoen waar je de data van wilt weten
    
    Returns
    ----------
    df : Pandas DataFrame
        Een dataframe waarin de data van het gegeven seizoen staat op volgorde van de behaalde punten
        
    """
    
    #Filteren voor het maken van een lijst met alle team namen in een league
    league_id = league[league['name']== league_naam].iloc[0,1]
    seizoen_filter = match[match['season'] == seizoen]
    team_ids = set(list(seizoen_filter[seizoen_filter['league_id'] == league_id]['home_team_api_id']))
    team_ids = list(team_ids)
    
    #Maken van een lijst met alle team namen in een league
    team_names = []
    for i in team_ids:
        team_names.append(team[team['team_api_id'] == i].iloc[0,3])
    
    #Aanmaken dataframe voor gebruik    
    df = statistiek(team_names[0],seizoen)

    #Loopen over de list met team namen om het dataframe vol te zetten
    print('stats season', seizoen, 'from', league_naam)
    for i in range(1, len(team_names)):
        df = pd.concat([df, statistiek(team_names[i], seizoen)])
        print(str(round(100/(len(team_names)+1)*i,2))+'%', load_screen[(i-1)%15], end = '\r')
    print('100.0% This adventure was brought to you by Group 3B.........')
    
    #Sorteren van het dataframe op basis van behaalde punten
    df = df.sort_values(by='punten', ascending = False).set_index('team')
    
    return df 

[Terug naar het begin](#begin)

## Het runnen van de functies <a name='run1'></a>
Hieronder worden de functies statistiek en ranglijst uitgevoerd:

In [ ]:
statistiek('Heracles Almelo', '2009/2010')

In [ ]:
ranglijst('Netherlands Eredivisie', '2009/2010')

#### Einde Sprint 2

[Terug naar het begin](#begin)

# Sprint 3 <a name='spr3'></a>

Voordat we aan de SQL functies beginnen, word er een decorator gemaakt om queries met SQL code te kunnen lezen.

In [ ]:
def sql_lezer(func):
    """
    Een decorator die een functie met een query omzet tot een Pandas DataFrame

    Parameters
    ----------
    func: Python Function
        Een python function die een query bevat

    Returns
    -------
    df : Pandas DataFrame
        Een DataFrame met het resultaat van de query.
    """
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        query = func(*args, **kwargs)
        
        #Aanmaken van try om te proberen de SQL query in te lezen.
        try:
            df = pd.read_sql(query, connect)
            return df
        
        #Aanmaken van except om error's bij het inlezen niet de 'run all' van jupyter te laten stoppen, deze geeft het type melding als tekst weer.
        except Exception as e:
            print(f"Error: {e}")
            return None
        
    return wrapper

[Terug naar het begin](#begin)

## Eigen functies: Prestaties van Heracles Almelo <a name='prestatie'></a>

In de volgende cellen code worden de functies gemaakt om te kijken naar de prestaties van ons team, Heracles Almelo. Eerst wordt er bekeken hoeveel wedstrijden er in elk seizoen zijn gespeeld.

### wedstrijden

In [ ]:
@sql_lezer
def wedstrijden(team_id):
    """
    Haalt het aantal gespeelde wedstrijden op van een bepaald team voor elk seizoen.

    Parameters
    ----------
    team_id : int
        Het team ID van het team waarvan het aantal gespeelde wedstrijden wordt opgehaald.

    Returns
    -------
    query : str
        Een SQL-query die het aantal gespeelde wedstrijden door het opgegeven team voor elk seizoen toont, 
        met de kolommen seizoen en Games_played. De query wordt gelezen en veranderd in een dataframe door
        de decorator: sql_lezer.
    """
    
    query = f"""
        SELECT season
                    , COUNT(DISTINCT(match_api_id)) AS Games_played
            FROM match
        
        WHERE (home_team_api_id = {team_id}) OR (away_team_api_id = {team_id})
        
        GROUP BY season
    """
    return query

[Terug naar het begin](#begin)

### wedstrijden_2010

Vervolgens wordt er gekeken naar het aantal gespeelde wedstrijden in het jaar 2010.

In [ ]:
@sql_lezer
def wedstrijden_2010(team_id):
    """
    Haalt het aantal gespeelde wedstrijden op van een bepaald team in het jaar 2010.

    Parameters
    ----------
    team_id : int
        Het team ID van het team waarvan het aantal gespeelde wedstrijden wordt opgehaald.

    Returns
    -------
    query : str
        Een SQL-query die het aantal gespeelde wedstrijden door het opgegeven team in het jaar 2010 toont, 
        met de kolommen seizoen en Games_played. De query wordt gelezen en veranderd in een dataframe door
        de decorator: sql_lezer.
    """
    
    query = f"""
        SELECT season
                    , COUNT(DISTINCT(match_api_id)) AS Games_played
            FROM match
        
        WHERE ((home_team_api_id = {team_id}) OR (away_team_api_id = {team_id})) 
        And (date BETWEEN '2010-01-01 00:00:00' AND '2010-12-31 00:00:00' )
        
        GROUP BY season

    """
    return query

[Terug naar het begin](#begin)

### maak_tabel

Hieronder wordt er een tabel aangemaakt om bij de volgende opgaven te gebruiken. Deze tabel wordt gebruikt om het aantal punten per seizoen per competitie te tonen.

In [ ]:
@sql_lezer
def maak_tabel(league_id):
    """
    Haalt de punten op van elk team per seizoen in de opgegeven competitie. Deze waarden worden vervolgens in een nieuwe tabel gezet.

    Parameters
    ----------
    league_id : int
        Het ID van de competitie waarvan de punten van elk team per seizoen worden opgehaald, ook wordt de plaats in de competities getoond.

    Returns
    -------
     query : str
        Een SQL-query die de punten van elk team per seizoen in de opgegeven competitie toont,
        met de kolommen league_id, season, Team_ID, Home_Points, Away_Points, Total_Points en Rank.
        De resultaten zijn gesorteerd op seizoen en totale punten in aflopende volgorde. De query wordt gelezen
        en toegepast door de decorator: sql_lezer.
    """
    
    query = f""" 
        CREATE TEMP TABLE pps_{league_id} AS
        SELECT h.league_id
                    , h.season
                    , h.Team_ID AS team_api_id
                    , h.Points AS punten_thuis
                    , a.Points AS punten_uit
                    , (h.Points + a.Points) AS punten_totaal
                    , ROW_NUMBER() OVER (PARTITION BY h.season ORDER BY (h.Points + a.Points) DESC) AS rang_seizoen
                
            FROM (SELECT league_id
                                    , season
                                    , home_team_api_id AS Team_ID
                                    , SUM(CASE WHEN (home_team_goal > away_team_goal) THEN 3
                                                          WHEN (home_team_goal = away_team_goal) THEN 1
                                                          WHEN (home_team_goal < away_team_goal) THEN 0
                                                          END) AS Points
                            FROM match
                            WHERE league_id = {league_id}
                            GROUP BY season
                                            , Team_ID) AS h
                        
        JOIN (SELECT league_id
                                , season
                                , away_team_api_id AS Team_ID
                                , SUM(CASE WHEN (away_team_goal > home_team_goal) THEN 3
                                                      WHEN (away_team_goal = home_team_goal) THEN 1
                                                      WHEN (away_team_goal < home_team_goal) THEN 0
                                                      END) AS Points
                    FROM match
                    WHERE league_id = {league_id}
                    GROUP BY season
                                    , Team_ID) AS a 
            ON h.season = a.season AND h.Team_ID = a.Team_ID
                    
        ORDER BY h.season, punten_totaal DESC;
    """
    return query.strip()

[Terug naar het begin](#begin)

### punten_per_seizoen

Nu wordt de functie punten_per_seizoen aangemaakt, die de aangemaakte tabel toont.

In [ ]:
@sql_lezer 
def punten_per_seizoen(league_id):
    """
    Deze functie toont de tabel met het league_id waarmee de punten tabel is aangemaakt
    
    Parameters
    ----------
    league_id : int
        Het ID van de competitie waarvan de tabel die is aangemaakt met maak_tabel() is aangemaakt.

    Returns
    ---------
     query : str
        Een SQL-query die is gemaakt met maak_tabel(). Deze bevat de volgende kolommen:
        league_id, season, Team_ID, Home_Points, Away_Points, Total_Points en Rank.
        De query wordt gelezen en veranderd in een dataframe door de decorator: sql_lezer.
    
    """
    
    query = f"""
        SELECT *
            FROM pps_{league_id}
    """
    return query

[Terug naar het begin](#begin)

### plaats_team

Als laatste wordt in de competitie gekeken naar plaats van Heracles Almelo per seizoen.

In [ ]:
@sql_lezer
def plaats_team(league_id, team_id):
    """
    Deze functie toont de tabel met het league_id waarmee de punten tabel is aangemaakt, deze is gefilterd op het aangegeven team_id.
    
    Parameters
    ----------
    league_id : int
        Het ID van de competitie waarvan de tabel die is aangemaakt met maak_tabel() is aangemaakt.

    Returns
    ---------
     query : str
        Een SQL-query die is gemaakt met maak_tabel(), gefilterd op het gegeven team_id. 
        Deze bevat de volgende kolommen: league_id, season, Team_ID, Home_Points, Away_Points, Total_Points en Rank.
        De query wordt gelezen en veranderd in een dataframe door de decorator: sql_lezer.
    """
    
    query = f"""
    SELECT *
        FROM pps_{league_id}
        
        WHERE team_api_id = {team_id}
    """
    return query

[Terug naar het begin](#begin)

## Bekijken van de prestaties <a name='prebekijk'></a>

In de onderstaande cellen zijn de resultaten in een cluster bij elkaar te vinden, de resultaten zijn afkomend van de vorige 5 eigen functies. Hierbij is voor het maken van de tabel pandas gebruikt en voor het selecteren word SQL gebruikt.

In [ ]:
wedstrijden(9791)

In [ ]:
wedstrijden_2010(9791)

In [ ]:
maak_tabel(13274)

De functie maak_tabel geeft als het goed is een Error 'NoneType' object op. Dit komt omdat de decorator nodig was om de functie te laten werken. Echter kan de decorator geen dataframe aanmaken doordat er een tabel wordt aangemaakt om te kunnen gebruiken voor de volgende twee functies.

In [ ]:
punten_per_seizoen(13274)

In [ ]:
plaats_team(13274, 9791)

[Terug naar het begin](#begin)

## Onderzoek teameigenschappen <a name='team'></a>

Nu is het tijd om te kijken naar welke impact verschillende teameigenschappen hebben voor het hoger scoren van punten. Hiervoor worden er eerst wat DataFrames aangepast en aangemaakt.

### kolom_seizoen <a name='kolmaken'></a>

Voordat er begonnen wordt met het onderzoek word er, met behulp van een eigen functie, een kolom aangemaakt om twee dataframes te kunnen mergen.

In [ ]:
def kolom_seizoen(df, col):
    """
    Geeft een lijst van seizoenen terug op basis van de maanden in een kolom van een pandas DataFrame.

    Parameters:
    -----------------
    df: Pandas DataFrame
        Het DataFrame waaruit de maanden worden gehaald
        
    col: str
        De naam van de kolom waaruit de maanden worden gehaald

    Returns:
    ------------
    season_list: list
        Een lijst van seizoenen in de vorm 'jaar/jaar', waarbij het jaar eindigt in juli.
    """
    
    #Aanmaken van een lege lijst om te gaan vullen 
    season_list = []
    
    #For-loop om de maanden om te zetten in seizoenen.
    for datum in df[col]:
        maand = datum[5:7]
        maand = int(maand)
        if maand > 7:
            season_list.append(datum[0:4] + '/' + str(int(datum[0:4]) + 1))
        else:
            season_list.append(str(int(datum[0:4]) - 1) + '/' + datum[0:4])
    
    return season_list

team_attributes['season'] = kolom_seizoen(team_attributes, 'date') 

### df_punten

Nu wordt er een dataframe gemaakt waarbij het aantal behaalde punten per team (van elke competitie) per seizoen getoond wordt. In de cel daaronder worden de twee dataframes team_attributes en df_points gemerged.

In [ ]:
def df_punten(seizoen):
    """
    Maakt het dataframe df_punten aan voor een merge. Hierin bevinden zich de kolommen: season, team_api_id en punten.

    Parameters:
    -----------------
    seizoen : str
        De jaartallen van het seizoen die bekeken wordt.

    Returns:
    ------------
    df_punten : Pandas DataFrame
        Een pandas DataFrame waarin het aangegeven seizoen, het team_id van de teams en het aantal behaalde punten staan.
    """
        
    #Zorgen dat het juiste seizoen wordt gekozen
    match_temp = match[match['season'] == seizoen].reset_index(drop=True)
    
    #Maak 2 lijsten voor de behaalde thuis en uit punten
    home_points = [3 if h > a else 1 if h == a else 0 for h, a in zip(match_temp['home_team_goal'], match_temp['away_team_goal'])]
    away_points = [3 if a > h else 1 if a == h else 0 for h, a in zip(match_temp['home_team_goal'], match_temp['away_team_goal'])]
    
    #Maak een lijst met daarin alle data die nodig is voor bij de merge (De ' \ ' is om door te gaan op de volgende regel)
    data = [[match_temp['season'][i], match_temp['home_team_api_id'][i], home_points[i]] for i in range(len(match_temp))] + \
               [[match_temp['season'][i], match_temp['away_team_api_id'][i], away_points[i]] for i in range(len(match_temp))]
    
    #Maken van de dataframe df_points
    df_punten = pd.DataFrame(data, columns=['season','team_api_id','punten'])

    #Groepeer op seizoen en team_id om de punten op te tellen
    df_punten = df_punten.groupby(['season','team_api_id']).sum().reset_index()
    
    return df_punten

In [ ]:
#merge team_attributes en df_points
team_attributes_points = pd.merge(team_attributes, df_punten('2009/2010'), on=['team_api_id', 'season'])

#verwijderen kolommen die te veel NaN-waardes hebben of nauwelijks invloed hebben op performance.
#De team_api_id en season kolommen zijn nog aanwezig om de date overzichtelijker te houden, deze hebben nauwelijks invloed op performance.
team_attributes_points = team_attributes_points.drop(['buildUpPlayDribbling', 'id', 'team_fifa_api_id', 'date'],axis=1)

display(team_attributes_points)

[Terug naar het begin](#begin)

## Visualisaties <a name='visu1'></a>

Voor de visualisaties wordt er gezocht naar correlatie tussen het puntenaantal en de teameigenschappen. Eerst worden de functies gemaakt en vervolgens zullen deze functies aangeroepen worden.

### heatmap

Het maken van de heatmap geeft een duidelijk overzicht van de verschillende kolommen met numerieke variablen.

In [ ]:
def heatmap(kolommen, corrnaam, df):
    """
    Deze functie maakt een heatmap aan, gebaseerd op een lijst aan kolommen en een gegeven dataframe
    
    Parameters:
    ---------------
    kolommen : list[str]
        Een lijst met numerieke kolommen waarvoor de correlatie tot 'corrnaam' wordt uitgerekend.
    
    corrnaam : str
        De naam van de kolom waarmee de lijst van kolommen wordt gecorreleerd
        
    df : Pandas DataFrame
        Het DataFrame waar de eerder vernoemde kolommen instaan
        
    Returns:
    ---------------
    None
        Er wordt geen waarde gereturned. Er word een plot gemaakt van de correlatie in vorm van een heatmap.
    """
    
    #Maken van een nieuw dataframe met de gewilde kolommen
    corr = df[kolommen + [corrnaam]]

    #Maken van correlatie df
    corr_hm = corr.corr()
    corr_hm_col = corr_hm[corrnaam]
    corr_hm_fini = corr_hm_col.drop(corrnaam)

    #Plotten van de grafiek
    sns.heatmap(pd.DataFrame(corr_hm_fini.sort_values(ascending=False)), cmap='coolwarm', annot=True)
    return plt.show()

### scatterplots

De scatterplots zijn er om de correlatie nogmaals in beeld te brengen, echter laten de scatterplots het ook nog toe om de categoriale variablen te tonen.

In [ ]:
#Maken van scatterplots voor elke kolom, t.o.v. de punten kolom
def scatterplots(df, kolom):
    """
    Maakt een aantal scatter plots gelijk aan het aantal kolommen in de Dataframe. De scatterplots zijn tegen één specifieke kolom opgezet, om de 
    correlatie te visualiseren.
    
    Parameters:
    ----------------
    df : Pandas DataFrame
        Een pandas DataFrame met kolommen waarvan je de correlaties wilt onderzoeken.
    
    kolom : str
        De naam van de kolom waarmee de correlatie berekend word, ten opzichte van de andere kolommen.
        
    Returns:
    ---------------
    None :
        In plaats van iets te returnen laat de functie meerdere scatterplots zien.
    """    
    #For-loop om elke kolom een eigen scatterplot te geven
    for col in df.columns:
        #if-statement om te voorkomen dat de parameter kolom ook een scatterplot krijgt
        if col != kolom:
            #Zetten van de grootte van de plot
            plt.subplots(figsize=(15, 5))
            
            #Maken en benoemen van de assen van een scatterplot
            plt.scatter(df[col], df[kolom])
            plt.xlabel(col)
            plt.ylabel(kolom)
            plt.title('{} vs. {}'.format(kolom, col))
            plt.show()

### boxplots

Als laatste word er gekeken via boxplots. Boxplots tonen de verdeling van de variablen, waardoor er duidelijker kan worden gekeken naar de variablen. Deze verdeling kan laten zien of er misschien optimale niveau's van aanvals of verdedigings statistieken zijn, in plaats van dat er naar de correlatie wordt gekeken.

In [ ]:
def boxplots(df, kolom):
    """
    Maakt een aantal box plots gelijk aan het aantal kolommen in de Dataframe. De box zijn tegen één specifieke kolom opgezet, om de 
    verdeling t.o.v. die kolom te tonen.
    
    Parameters:
    ----------------
    df : Pandas DataFrame
        Een pandas DataFrame met kolommen waarvan je de verdeling wilt onderzoeken.
    
    kolom : str
        De naam van de kolom waarbij de verdeling bekeken word, ten opzichte van de andere kolommen.
        
    Returns:
    ---------------
    None :
        In plaats van iets te returnen laat de functie meerdere boxplots zien.
    """    
    #For-loop om elke kolom een eigen boxplot te geven
    for col in df.columns:
        #if-statement om te voorkomen dat de kolom-parameter ook een boxplot krijgt
        if col != kolom:
            #Zetten van de grootte van de plot
            plt.subplots(figsize=(15, 5))
            
            #Maken en benoemen van de assen van de boxplots
            sns.boxplot(x=col, y=kolom, data=df, color= 'skyblue')
            plt.xlabel(col)
            plt.ylabel(kolom)
            plt.title('{} vs. {}'.format(kolom, col))
            plt.show()

## Tonen van de grafieken <a name='run3'></a>

Hieronder tonen we van elke functie de resulterende grafieken. Ook word er een nieuw df en een list aangemaakt om deze te gebruiken voor de visualisaties.

In [ ]:
#Relevante kolommen met de hand kiezen voor correlatie (numerieke kolommen), kolommen met te veel missende waardes zijn ook weggelaten.
kolom_num = ['buildUpPlaySpeed', 'buildUpPlayPassing',
                   'chanceCreationPassing', 'chanceCreationCrossing', 'chanceCreationShooting',
                   'defencePressure', 'defenceAggression', 'defenceTeamWidth']

In [ ]:
heatmap(kolom_num, 'punten', team_attributes_points)

Bij de heatmap valt er te zien dat de correlaties niet erg hoog zijn, door de scatterplots en boxplots wordt er gehoopt om een beter begrip te krijgen over welke waardes impact hebben op het aantal gescoorde punten.

In [ ]:
#Weghalen van bepaalde kolommen die geen impact hebben op het aantal punten
tap = team_attributes_points.copy()
tap = tap.drop(['season', 'team_api_id'], axis = 1)

In [ ]:
scatterplots(tap, 'punten')

In [ ]:
boxplots(tap, 'punten')

Zo als te zien is uit de verschillende plots, valt er niet op 1 statistiek te drukken. Verschillende statistieken tonen betere resultaten op verschillende getallen, van zowel hoog, middel en laag. Dit laat zien dat de correlatie misschien niet hoog is wegens de brede spreiding van statistieken. De teameigenschappen zouden verder onderzocht moeten worden om goed te kunnen vertellen welke combinaties in statistiek beter werken dan anderen.

[Terug naar het begin](#begin)